In [1]:
from sec_edgar_downloader import Downloader

dl = Downloader("MyCompanyName", "my.email@domain.com")

In [2]:
dl.get("10-K", "MSFT", limit=1)

1

In [1]:
import requests
import pandas as pd
from typing import List, Dict
import yfinance as yf
import sec_edgar_downloader as sec

def lookup_company_info(isins: List[str]) -> List[Dict]:
    """
    Retrieve company information for a list of ISINs.
    
    Parameters:
    isins (List[str]): List of International Securities Identification Numbers (ISINs)
    
    Returns:
    List[Dict]: List of dictionaries containing company information
    """
    company_info_list = []
    
    for isin in isins:
        company_info = {
            'ISIN': isin,
            'Company Name': None,
            'Website': None,
            'CIK': None
        }
        
        try:
            # First, try to get info using yfinance
            ticker_symbol = None
            
            # Attempt to convert ISIN to ticker
            try:
                ticker = yf.Ticker(isin)
                info = ticker.info
                
                # Extract company name
                if 'longName' in info and info['longName']:
                    company_info['Company Name'] = info['longName']
                
                # Extract website
                if 'website' in info and info['website']:
                    company_info['Website'] = info['website']
                
                # Try to get ticker symbol
                ticker_symbol = ticker.ticker
            except Exception as yf_error:
                print(f"YFinance lookup failed for {isin}: {yf_error}")
            
            # If ticker is available, try SEC Edgar lookup
            if ticker_symbol:
                try:
                    # Initialize SEC Edgar downloader
                    dl = Downloader("MyCompanyName", "my.email@domain.com")
                    
                    # Get company information from SEC
                    company_info['CIK'] = dl.get_cik_by_ticker(ticker_symbol)
                except Exception as sec_error:
                    print(f"SEC Edgar lookup failed for {ticker_symbol}: {sec_error}")
            
            company_info_list.append(company_info)
        
        except Exception as e:
            print(f"Error processing ISIN {isin}: {e}")
            company_info_list.append(company_info)
    
    return company_info_list

def main():
    # Example list of ISINs
    example_isins = [
        'US0378331005',  # Apple
        'US5949181045',  # Microsoft
        'US88160R1014'   # Tesla
    ]
    
    # Get company information
    results = lookup_company_info(example_isins)
    
    # Convert to DataFrame for easy viewing
    df = pd.DataFrame(results)
    print(df)
    
    # Optional: Save to CSV
    df.to_csv('company_info.csv', index=False)

In [6]:
main()

SEC Edgar lookup failed for AAPL: 'Downloader' object has no attribute 'get_cik_by_ticker'
SEC Edgar lookup failed for MSFT: 'Downloader' object has no attribute 'get_cik_by_ticker'
SEC Edgar lookup failed for TSLA: 'Downloader' object has no attribute 'get_cik_by_ticker'
           ISIN           Company Name                    Website   CIK
0  US0378331005             Apple Inc.      https://www.apple.com  None
1  US5949181045  Microsoft Corporation  https://www.microsoft.com  None
2  US88160R1014            Tesla, Inc.      https://www.tesla.com  None


In [2]:
import requests
import json

def get_cik_by_ticker(ticker: str) -> str:
    """
    Retrieve CIK (Central Index Key) for a given stock ticker.
    
    Args:
        ticker (str): Stock ticker symbol
    
    Returns:
        str: CIK number or None if not found
    """
    # SEC EDGAR company tickers mapping file
    tickers_url = "https://www.sec.gov/files/company_tickers.json"
    
    try:
        # Fetch the tickers JSON
        response = requests.get(tickers_url, headers={
            'User-Agent': 'Your Name your.email@example.com'
        })
        
        # Raise an exception for bad responses
        response.raise_for_status()
        
        # Parse the JSON response
        tickers_data = response.json()
        
        # Normalize ticker to uppercase
        ticker = ticker.upper()
        
        # Search for the CIK
        for key, company_info in tickers_data.items():
            if company_info['ticker'] == ticker:
                # Pad CIK with zeros to ensure 10 digits
                return str(company_info['cik_str']).zfill(10)
        
        # If ticker not found
        return None
    
    except Exception as e:
        print(f"Error retrieving CIK for {ticker}: {e}")
        return None

def get_company_info_by_cik(cik: str) -> dict:
    """
    Retrieve basic company information using CIK.
    
    Args:
        cik (str): CIK number
    
    Returns:
        dict: Company information
    """
    # Ensure CIK is padded to 10 digits
    cik = str(cik).zfill(10)
    
    # Construct SEC EDGAR company information URL
    url = f"https://data.sec.gov/submissions/CIK{cik}.json"
    
    try:
        # Fetch company information
        response = requests.get(url, headers={
            'User-Agent': 'Your Name your.email@example.com'
        })
        
        # Raise an exception for bad responses
        response.raise_for_status()
        
        # Parse the JSON response
        company_data = response.json()
        
        # Extract relevant information
        return {
            'name': company_data.get('name', 'N/A'),
            'ticker': company_data.get('tickers', ['N/A'])[0],
            'sic': company_data.get('sic', 'N/A'),
            'website': company_data.get('website', 'N/A')
        }
    
    except Exception as e:
        print(f"Error retrieving company info for CIK {cik}: {e}")
        return {}

def main():
    # Example usage
    tickers = ['AAPL', 'MSFT', 'GOOGL']
    
    for ticker in tickers:
        # Get CIK
        cik = get_cik_by_ticker(ticker)
        
        if cik:
            print(f"\nTicker: {ticker}")
            print(f"CIK: {cik}")
            
            # Get additional company info
            company_info = get_company_info_by_cik(cik)
            
            # Print company details
            for key, value in company_info.items():
                print(f"{key.capitalize()}: {value}")
        else:
            print(f"Could not find CIK for {ticker}")

In [10]:
main()


Ticker: AAPL
CIK: 0000320193
Name: Apple Inc.
Ticker: AAPL
Sic: 3571
Website: 

Ticker: MSFT
CIK: 0000789019
Name: MICROSOFT CORP
Ticker: MSFT
Sic: 7372
Website: 

Ticker: GOOGL
CIK: 0001652044
Name: Alphabet Inc.
Ticker: GOOGL
Sic: 7370
Website: 


In [11]:
dat = yf.Ticker("MSFT")
dat.info

{'address1': 'One Microsoft Way',
 'city': 'Redmond',
 'state': 'WA',
 'zip': '98052-6399',
 'country': 'United States',
 'phone': '425 882 8080',
 'website': 'https://www.microsoft.com',
 'industry': 'Software - Infrastructure',
 'industryKey': 'software-infrastructure',
 'industryDisp': 'Software - Infrastructure',
 'sector': 'Technology',
 'sectorKey': 'technology',
 'sectorDisp': 'Technology',
 'longBusinessSummary': 'Microsoft Corporation develops and supports software, services, devices and solutions worldwide. The Productivity and Business Processes segment offers office, exchange, SharePoint, Microsoft Teams, office 365 Security and Compliance, Microsoft viva, and Microsoft 365 copilot; and office consumer services, such as Microsoft 365 consumer subscriptions, Office licensed on-premises, and other office services. This segment also provides LinkedIn; and dynamics business solutions, including Dynamics 365, a set of intelligent, cloud-based applications across ERP, CRM, power 

In [4]:
!conda install -c conda-forge yfinance=0.2.53 -y

Solving environment: unsuccessful initial attempt using frozen solve. Retrying with flexible solve.
Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 25.1.1
  latest version: 25.3.0

Please update conda by running

    $ conda update -n base -c conda-forge conda

Or to minimize the number of packages updated during conda update use

     conda install conda=25.3.0



## Package Plan ##

  environment location: /opt/anaconda3/envs/scrapegraph

  added / updated specs:
    - yfinance=0.2.53


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    yfinance-0.2.53            |     pyhd8ed1ab_0          89 KB  conda-forge
    ------------------------------------------------------------
                                           Total:          89 KB

The following packages will be DOWNGRADED:

  yfinance                              0.2.54-pyhd8ed1ab_0 

In [1]:
ISIN = ['US9884981013','US91913Y1001','US1266501006','US23331A1097',
'US05722G1004','US4278661081','US0718131099','US9694571004','US8550301027',
'US0028241000','US69351T1060','US7445731067','US3703341046',
'US2358511028','US02209S1033','US7865142084','US85590A4013',
'US55616P1049','US25746U1097','US1491231015','US30161N1019',
'US0091581068','US3695501086','US14149Y1082','US4385161066',
'US0970231058','US74834L1008','US6819191064','US42809H1077',
'CH0048265513','US29364G1031','US7617131062','US1344291091']

In [2]:
import yfinance as yf

In [5]:
for i in ISIN:
    ticker = yf.Ticker(i)
    try:
        print(ticker.info)
    except:
        print(f"ISIN {i} delisted")

AttributeError: module 'requests.cookies' has no attribute 'clear'

In [3]:
import investpy

df = investpy.stocks.search_stocks(by='isin', value=ISIN[1])

In [4]:
from time import sleep
from tqdm.notebook import tqdm
list_df = []
delisted = []

for i in tqdm(ISIN):
    try:
        df = investpy.stocks.search_stocks(by='isin', value=i)
        list_df.append(df)
    except:
        delisted.append(i)
        print(f"ISIN {i} delisted")
    sleep(0.5)

  0%|          | 0/33 [00:00<?, ?it/s]

ISIN US8550301027 delisted
ISIN US7865142084 delisted
ISIN US85590A4013 delisted
ISIN US7617131062 delisted


In [6]:
list_df

[         country         name        full_name          isin currency symbol
 0  united states  Yum! Brands  Yum! Brands Inc  US9884981013      USD    YUM,
          country           name                  full_name          isin  \
 0         mexico  Valero Energy         Valero Energy Corp  US91913Y1001   
 1  united states  Valero Energy  Valero Energy Corporation  US91913Y1001   
 2        germany  Valero Energy         Valero Energy Corp  US91913Y1001   
 
   currency symbol  
 0      MXN    VLO  
 1      USD    VLO  
 2      EUR    V1L  ,
          country             name        full_name          isin currency  \
 0          chile  CVS Health Corp  CVS Health Corp  US1266501006      USD   
 1         mexico  CVS Health Corp  CVS Health Corp  US1266501006      MXN   
 2  united states  CVS Health Corp  CVS Health Corp  US1266501006      USD   
 3        germany  CVS Health Corp  CVS Health Corp  US1266501006      EUR   
 
   symbol  
 0    CVS  
 1    CVS  
 2    CVS  
 3    CV

In [8]:
import pandas as pd

In [10]:
cdf = pd.concat(list_df)

In [15]:
cdf.loc[cdf['country']=='united states','isin'].unique()
# some ISINs may not be in the united states

array(['US9884981013', 'US91913Y1001', 'US1266501006', 'US23331A1097',
       'US05722G1004', 'US4278661081', 'US0718131099', 'US9694571004',
       'US0028241000', 'US69351T1060', 'US7445731067', 'US3703341046',
       'US2358511028', 'US02209S1033', 'US55616P1049', 'US25746U1097',
       'US1491231015', 'US30161N1019', 'US0091581068', 'US3695501086',
       'US14149Y1082', 'US4385161066', 'US0970231058', 'US74834L1008',
       'US6819191064', 'US42809H1077', 'CH0048265513', 'US29364G1031',
       'US1344291091'], dtype=object)